# Chapter 10 - Influential observations diagnostics

Joshua French

To open this information in an interactive Colab notebook, click the Open in Colab graphic below.

<a href="https://colab.research.google.com/github/jfrench/LinearRegression/blob/master/notebooks/10-influential-observations-diagnostics-notebook.ipynb"> <img src="https://raw.githubusercontent.com/jfrench/LinearRegression/1584e3a0a89ce40e06a90dd7c043d270042fc015/images/qr-influential-observations.svg"> </a>

------------------------------------------------------------------------

In [ ]:
if(!require(api2lm, quietly = TRUE)) {
  install.packages("api2lm",
                   repos = "https://cran.rstudio.com/")
  library(api2lm)
}

We adjust some printing options for clarity.

In [ ]:
options(digits = 5, scipen = 2)

# Influential observations

One of the key assumptions we implicitly make when we fit a regression model to data is that the fit of the model isn’t dramatically influenced by a few observations.

-   Each observation should have a similar impact on the fitted model.

We want our model fit to behave like a democracy (each observation has an equal say in the fit) instead of a dictatorship or oligarchy (only a few observations, at most, have real power in determining the fit of the model to the data).

An **influential observation** dramatically changes the fitted model based on whether it is included or excluded from the analysis.

-   An observation may not be influential on its own but can be influential when working with other observations.

Influential observations are often extreme with respect to their response values or their associated predictors.

-   An **outlier** is an observation with a large residual.
    -   The response is relatively far from its fitted value.
-   A **leverage point** is an observation whose predictor values are extreme relative to most other observations.
    -   A person with “leverage” over another person has the power to influence what that person does.
    -   A leverage point has the ability to exert strong influence over a model fit.

Outliers, leverage points, and influential observations are all model specific.

-   An outlier (etc.) may be an outlier for one fitted model but not for a different one.

In what follows, we assume that our fitted model has $p$ regression coefficients to estimate.

# `home_sales` data

To facilitate our discussion, we will use the `home_sales` data set in the **api2lm** package, which contains information about homes sold in King County, WA between May 2014 and May 2015. The data set is a data frame with 216 rows and 8 columns:

-   `price`: sale price (log10 units, `numeric`).
-   `bedrooms`: number of bedrooms (`numeric`).
-   `bathrooms`: number of bathrooms (`numeric`).
-   `sqft_living`: size of living space (`numeric`).
-   `sqft_lot`: size of property (`numeric`).
-   `floors`: number of floors (`numeric`).
-   `waterfront`: a factor variable with levels `"yes"` and `"no"` that indicate whether a home as a waterfront view (`factor`).
-   `condition`: a factor variable indicating the condition of a house with levels ranging from `"poor"` to `"very good"` (`factor`).

We load the data below.

In [ ]:
data(home_sales, package = "api2lm")

The validity of the assumptions we want to check depend on the model fit to the data.

We will regress the `price` variable on all of the remaining variables in `home_sales`.

In [ ]:
lmod <- lm(price ~ ., data = home_sales)
coef(lmod)

We use the `summary` function to provide a numeric summary of the data set for later use.

In [ ]:
summary(home_sales)

# LOO and studentized residuals

Leave-one-out (LOO) and studentized residuals are specific types of residuals that are useful for identifying outliers and influential observations.

LOO residuals are directly connected with LOO cross-validation.

-   LOO residuals are also known as deleted residuals.

Let $\hat{Y}_{i(i)}$ denote the prediction of observation $i$ for the model that was fit using all $n$ observations except the $i$th one.

-   This is the predicted response for observation $i$ using LOO cross-validation.

The **leave-one-out residual** for observation $i$ is defined as

$$l_i = Y_i - \hat{Y}_{i(i)}.$$

If an observation is influential, then the LOO residual may be substantially larger in magnitude than the ordinary residual.

-   An influential observation is like a black hole that pulls the fitted model toward itself, so its associated residual might be small.

Interestingly, the $i$th LOO residual can be computed via the equation

$$
l_i = \frac{\hat{\epsilon}_i}{1-h_i},
$$ where $h_i=\mathbf{H}_{i,i}$ is the $i$th diagonal element of the hat matrix given by $\mathbf{H} = \mathbf{X}(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T$.

-   We do not need to fit multiple models to compute the the LOO residuals.

Let $\hat{\sigma}_{(i)}^2$ denote the residual variance for the model fit using all $n$ observations except observation $i$.

-   We will call this the **LOO residual variance**.

The **studentized residual** for observation $i$ is defined as

$$
t_i = \frac{\hat{\epsilon_i}}{\hat{\sigma}_{(i)}\sqrt{1-h_i}}.
$$

Assuming the standard linear model assumptions are true, then each studentized residual will have a $t_{n-p-1}$ distribution.

The means that studentized residuals are identically distributed (which means they have the same variance).

However, the studentized residuals are not independent.

The `rstudent` function extracts the studentized residuals from an `lm` object.

We summarize the studentized residuals for the model fit to the `home_sales` data below. There are some relatively extreme studentized residuals with values below -3 and above 3.

In [ ]:
stud_resid <- rstudent(lmod)
summary(stud_resid)

# Outliers

We describe two common approaches for identifying outliers.

The simplest approach for identifying outliers is to use an index plot of the studentized residuals to determine the observations with extreme residuals.

-   An index plot is a plot of a set of statistics versus their observation number.
-   We will create a scatter plot of the pairs $(i, t_i)$ for $i=1,2,\ldots,n$.

The other approach is to compare the set of studentized residuals to the appropriate quantile of a $t_{n-p-1}$ distribution.

To evaluate whether a *single* observation is an outlier, we compare the absolute value of its studentized residual to $t^{\alpha/2}_{n-p-1}$, i.e., the $1-\alpha/2$ quantile of a $t$ distribution with $n-p-1$ degrees of freedom.

-   If $|t_i| > t^{\alpha/2}_{n-p-1}$, then observation $i$ is an outlier.
-   We won’t do this because we have more than one observation to evaluate.

To identify the outliers from the $n$ observations of our fitted model, we should adjust the previous idea using the Bonferroni correction to address the multiple comparisons problem.

-   We conclude observation $i$ is an outlier if $|t_i| > t^{\alpha/2n}_{n-p-1}$.

## Outlier example

We use the methods described above to identify the outliers of the model we fit to the `home_sales` data.

We first create an index plot of the studentized residuals based on the fitted model `lmod`.

In [ ]:
plot(rstudent(lmod), ylab = "studentized residuals")

We see that there are a few outlying observations, though it is difficult to identify them.

The `outlier_plot` function from the **api2lm** package offers a more convenient way to identify outliers using an index plot of the studentized residuals.

The main arguments of the `outlier_plot` function are:

-   `model`: the fitted model.
-   `id_n`: the number of points to identify with labels.
-   `add_reference`: a logical value indicating whether the Bonferroni-corrected $t$ quantiles ($t^{1-\alpha/2n}_{n-p-1}$ and $t^{\alpha/2n}_{n-p-1}$) are added as reference lines to the plot. The default is `TRUE`.
-   `alpha`: the $\alpha$ value used for the $t$ quantiles. The default is `0.05`.

We use the `outlier_plot` function to identify the 2 most extreme outliers for our fitted model.

-   Observations 214 and 33 have the most extreme studentized residuals.
-   Observation 214 is more extreme than the Bonferroni-adjusted $t$ quantile.

In [ ]:
outlier_plot(lmod, id_n = 2)

The `outlier_test` function from the **api2lm** package can be used to identify the observations with studentized residuals more extreme than the appropriate Bonferroni-corrected $t$ quantiles. The main arguments to the `outlier_test` function are:

-   `model`: the fitted model.
-   `n`: the number of outliers to return (by default, all of them).
-   `alpha`: the desired family-wise error rate for the family of $n$ tests. The default is `alpha = 0.05`.

Running `outlier_test` on a fitted model will return the identified outliers and their Bonferroni-adjusted p-values (`adj_pvalue`). If the adjusted p-value is less than `alpha`, then the observation is declared an outlier.

We use `outlier_test` to see that observation 214 is an outlier according to this test.

In [ ]:
outlier_test(lmod)

We examine the data for observation 214 in the `home_sales` data set.

In [ ]:
home_sales[214, ]

We see that it was sold for a (relatively) high price (top 5%) and appears to be a large home with respect to living space and lot size.

In fact, if we compare the predictor values for this house to the numeric summary fo `home_sales` that we computed earlier, we see that this is the largest house in the data set in terms of `sqft_living`!

In [ ]:
mean(home_sales$price >= 6.3579)

# Leverage Points

The **leverage values** are the diagonal elements of the hat matrix $\mathbf{H}=\mathbf{X}(\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T$.

The $i$th leverage value is given by $h_i=\mathbf{H}_{i,i}$, the $i$th diagonal position of the hat matrix.

The `hatvalues` function extracts the leverage values from an `lm` object.

The leverage values of a fitted model have the following two properties:

-   $0\leq h_i \leq 1$ for $i=1,2,\ldots,n$.
-   $\sum_{i=1}^n h_i = p$.

An observation is declared to be a leverage point if its leverage value is unusually large.

Kutner et al. (2005) suggest two different thresholds for identifying a leverage point.

-   Observation $i$ is declared to be an outlier if $h_i > 2p/n$.
    -   This approach can be overly sensitive.
-   Observation $i$ is declared to be an outlier if $h_i > 0.5$.

Alternatively, we can make a relative comparison of the leverage values to determine the ones that are unusually large relative to the rest.

## Leverage example

We seek to identify the leverage points of the `home_sales` data based on our fitted model.

The `leverage_plot` function from the **api2lm** package creates an index plot of the leverage values.

The main arguments of the `leverage_plot` function are:

-   `model`: the fitted model.
-   `id_n`: the number of points to identify with labels.
-   `ttype`: the threshold type. The default is `"half"`, which uses a reference threshold of `0.5`. Alternatively, we can choose `"2mean"`, which uses a reference threshold of $2p/n$.

We use the `leverage_plot` function to identify leverage points for the model fit to the `home_sales` data.

In [ ]:
leverage_plot(lmod)

Observations 33 and 179 are leverage points with leverage values greater than 0.5. Observation 214 has a leverage value nearly at 0.5, so we should consider it a leverage point.

If we examine the predictor values for these observations, we see observations with extremely large numbers of bedrooms, bathrooms, living space, or lots.

In [ ]:
home_sales[c(33, 179, 214), ]

# Influential Observations

An **influential observation** dramatically changes the fitted model based on whether it is included or excluded from the analysis.

-   An outlier with high leverage is most likely to be an influential observation.

Measures of influence are often related to various LOO statistics. Examples include:

-   The LOO residuals, $l_i = Y_i - \hat{Y}_{i(i)}$, $i = 1,2,\ldots,n$.
-   The LOO residual variances, $\hat{\sigma}^2_{(i)}$, $i=1,2,\ldots,n$.

We will now discuss several other measures of influence.

## DFBETA and DFBETAS

The **DFBETA** measure of influence is an $n\times p$ matrix of statistics that measures the change in the estimated regression coefficients when we fit the model using all $n$ observations versus when we fit the model leaving one observation out at a time.

The $i$th row of DFBETA is $$
\text{DFBETA}_i = \hat{\boldsymbol{\beta}} - \hat{\boldsymbol{\beta}}_{(i)}.
$$

-   $\hat{\boldsymbol{\beta}} = [\hat{\beta}_0, \hat{\beta}_1, \ldots, \hat{\beta}_{p-1}]$ is the vector of estimated regression coefficients when using all $n$ observations.
-   $\hat{\boldsymbol{\beta}}_{(i)} = [\hat{\beta}_{0(i)}, \hat{\beta}_{1(i)}, \ldots, \hat{\beta}_{p-1(i)}]$ is the vector of estimated regression coefficients obtained from the model fit using all $n$ observations except the $i$th observation.
-   $\text{DFBETA}_{ij} = \hat{\beta}_j - \hat{\beta}_{j(i)}$.

Identifying influential observations using the DFBETA matrix can be difficult because the sampling variance associated with each coefficient can be very different.

The **DFBETAS** matrix is a transformation of the DFBETA matrix that has been scaled so that the individual statistics have similar sampling variability.

DFBETAS is also an $n\times p$ matrix of statistics with the $j$th element of the $i$th row being defined as

$$\text{DFBETAS}_{i,j} = \frac{\hat{\beta}_{j-1} - \hat{\beta}_{j-1(i)}}{\hat{\mathrm{se}}(\hat{\beta}_{j-1})},\quad i=1,2,\ldots,n, \quad j=0,1,\ldots,p-1.$$

A DFBETAS statistic less than -1 or greater than 1 is often used as an indicator that the associated observation is influential in determining the fitted model.

One way of identifying influential observations is to create index plots the DFBETA or DFBETAs values for each regression coefficient and determining the estimates that substantially change when observation $i$ is excluded from analysis.

-   We recommend using index plots of the DFFBETAS values instead of the DFBETA values because the plots will have more similar scales and are easier to work with.

The `dfbeta` and `dfbetas` functions can be used to extract the DFBETA and DFBETAS matrices, respectively, from an `lm` object.

## DFBETAS example

The `dfbetas_plot` function in the **api2lm** creates index plots of the DFBETAS statistics for each regression coefficient.

The main arguments to the `dfbetas_plot` function are:

-   `model`: the fitted model.
-   `id_n`: the number of observations for which to print an associated label.
-   `regressors`: a formula indicating the regressors for which we want to plot the DFBETAS statistics. By default, index plots are created for all regressors.

Reference lines are automatically added at $\pm 1$ to indicate observations whose DFBETAS values are particularly extreme.

We plot the `bedrooms`, `bathrooms`, `sqft_living`, and `sqft_lot` variables for the fitted model of our `home_sales` data and label the two most extreme observations.

In [ ]:
dfbetas_plot(lmod, id_n = 2,
             regressors = ~ bedrooms + bathrooms + sqft_living + sqft_lot)

We can see from these plots that:

-   The estimated coefficient for `bedrooms` changes by more than 3 standard errors based on whether observation 179 is used in fitting the model.
-   The estimated coefficients for `sqft_living` and `sqft_lot` change by more than 1 standard error based on whether observations 33 or 214 are used in fitting the model.

## DFFITS

Welsch and Kuh (1977) proposed measuring an observation’s influence through the DFFITS statistic, which is the difference between its fitted value and its LOO predicted response value.

The **DFFITS** statistic for observations $i$ defined as

$$\text{DFFITS}_i = \hat{Y}_i - \hat{Y}_{i(i)}.$$

-   A large difference between a fitted value and its associated LOO predicted value provides evidence an observation is influential.

Belsley, Kuh, and Welsch (2005) suggest that observation $i$ is influential if $|\text{DFFITS}_i|>2\sqrt{p/n}$.

An index plot of the DFFITS statistics for all observations will indicate the observations most influential in changing their associated predicted value.

The `dffits` function extracts the DFFITS statistics for each observation from an `lm` object.

## DFFITS example

The \``dffits_plot` function creates an index plot of the DFFITS statistics from a fitted model.

The main arguments to the `dffits_plot` function are:

-   `model`: the fitted model.
-   `id_n`: the number of observations for which to print an associated label.

Horizontal reference lines are automatically added at $\pm 2\sqrt{p/n}$ to identify influential observations.

We create an index plot of the DFFITS statistics below. We identify the observations with the 3 most extreme DFFITS statistics.

In [ ]:
dffits_plot(lmod, id_n = 3)

We see that observations 33, 179, and 214 all have particularly extreme DFFITS statistics. Several other observations have statistics more extreme than $\pm 1$ (run `dffits_test(lmod)` for a complete list).

We have already discussed how unusual these observations are with respect to a combination of their response and predictor values.

## Cook’s distance

Cook (1977) proposed the **Cook’s distance** to summarize the potential influence of an observation with a single statistic.

The Cook’s distance for the $i$th observation is

$$D_i=\frac{\sum_{k=1}^n (Y_k - \hat{Y}_{k(i)})^2}{p \widehat{\sigma}^2} = \frac{1}{p} r_i^2 \frac{h_i}{1-h_i}.$$

-   $\hat{Y}_{k(i)}$ is the predicted response for observation $k$ when observation $i$ is not included in the analysis (this is a type of LOO prediction).
-   The **standardized residual** for observation $i$ is

$$
r_i = \frac{\hat{\epsilon}_i}{\widehat{\sigma}\sqrt{1-h_i}}.
$$

Cook’s distance blends information about whether an observation both an outlier and a leverage point into a single statistic.

The Cook’s distance increases in magnitude if the observation is an outlier since the standardized residual, $r_i$, will become large.

The Cook’s distance increases in magnitude if the observation is a leverage point since $h_i/(1-h_i)$ will become large.

Unlike the DFFITS statistic, the Cook’s distance quantifies the amount of influence potential an observation has across all $n$ fitted values instead of only a single fitted value (Kutner et al. 2005).

An index plot of the Cook’s distances can be used to identify observations with unusually large Cook’s distances.

-   The `cooks.distance` function extracts the Cook’s distance values from an `lm` object.
-   Kutner et al. (2005) suggest declaring an observation to be influential if its Cook’s distance is more than $F^{0.5}_{p, n-p}$, i.e., the 0.5 quantile of an $F$ distribution with $p$ numerator degrees of freedom and $n-p$ denominator degrees of freedom.

## Cook’s distance example

The `cooks_plot` function from the **api2lm** package creates an index plot of the Cook’s distances.

The main arguments to the `cooks_plot` function are:

-   `model`: the fitted model.
-   `id_n`: the number of observations for which to print an associated label.

A horizontal reference line is automatically added at $F^{0.5}_{p,n-p}$ to identify influential observations.

We create an index plot of the Cook’s distances below. We identify the observations with the 3 most extreme Cook’s distances.

In [ ]:
cooks_plot(lmod, id_n = 3)

We see that observations 33, 179, and 214 all have particularly extreme Cook’s distances, indicating those observations are influential in some sense. This is consistent with our previous results.

## Influence plots

An **influence plot** is a scatter plot of the studentized or standardized residuals versus the leverage values. The size of each point in the plot is proportional to either its Cook’s distance or DFFITS statistic.

In general, observations that are extreme with respect to their residuals, leverage values, or Cook’s distance/DFFITS statistics are more likely be be potentially influential.

## Influence plot example

The `influence_plot` function from the **api2lm** package creates an influence plot for an `lm` object.

The main arguments to the `influence_plot` function are:

-   `model`: the fitted model.
-   `rtype`: the type of residual to plot on the y-axis. The default is the studentized residuals.
-   `criterion`: the criterion that controls the size of the points. The default is the Cook’s distance.
-   `id_n`: the number of observations for which to print an associated label. The most extreme observations with respect to the criterion are labeled.

We create in influence plot for the model fit to the `home_sales` data below.

In [ ]:
influence_plot(lmod)

Similar to our previous results, we see that observations 33, 179, and 214 are extreme with respect to characteristics like their residual, leverage, or Cook’s distance.

# Reaction

How should we react when we find influential observations?

It is best to work through a series of diagnostic steps to decide what to do.

**Should the influential observations be in the data set?**

First, are the influential observations in the data set mistakenly?

Is an observation influential because it is a data entry error?

-   If it is, then correct the error if possible.
-   If the data entry error cannot be corrected, then it is appropriate to remove the observation from our analysis. We must make sure to mention this in the discussion of our results.

Is the influential observation part of the population of interest?

-   If we are studying dogs, but the influential observation is a cat, then it is okay to remove the observation from our analysis. We must make sure to mention this in the discussion of our results.
-   More generally, there may be a subtle but compelling reason why an observation isn’t part of the population of interest.

If there isn’t a compelling reason to exclude an observation from our analysis except that it seems to be influential, then we should keep the observation in our analysis.

Consider observations 33, 179, and 214 from our `home_sales` data set, which we have identified as being potentially influential.

In [ ]:
home_sales[c(33, 179, 214),]

Observation 179 has 33 bedrooms but only 1.75 bathrooms and 1,620 square feet of living space. This seems unlikely and there is likely a data entry error for that observation (perhaps it should be 3 bedrooms). Since we don’t have any details about this specific sale, we could reasonably exclude this observation from our analysis.

Observations 33 and 214 are houses with massive lot sizes of over 7 acres. In fact, except for observation 141, which has a lot size of 223,462 square feet, observations 33 and 214 have lot sizes that are more than 5 times as large as any other observation. We could argue that these observations are not representative of a typical home sold in King County during this time period. It *could* be appropriate to remove these observations from our analysis if we are convinced they aren’t part of the target population we want to know about.

**Are our goals affected?**

Second, after making sure an influential observation isn’t mistakenly in our data set, we should ask ourselves whether an influential observation practically affects the goals of our analysis.

If the purpose of our analysis is to identify the general relationship between a predictor and response after adjusting for some other variables, then perhaps the influential observations don’t practically change our results.

-   Does the estimated coefficient change from significant to insignificant?
-   Does the sign of the coefficient change based on whether the observation is included?

If the answer to these last two questions is no, then perhaps the influential observations are not problematic.

Let’s fit the same model to the `home_sales` data without observations 33, 179, and 214.

In [ ]:
lmod2 <- lm(price ~ ., data = home_sales[-c(33, 179, 214),])

We use the `coef_compare` function from the **api2lm** package to compare the estimated coefficients of our original fitted model to the new model as well as compute their percentage change.

In [ ]:
coef_compare(lmod, lmod2)

When we remove these 3 influential observations, the estimated `bedrooms` coefficient changed by nearly 800% and the estimated standard error by nearly 200%.

The sign of the coefficient also flips from positive to negative, which certainly goes against our intuition.

However, the `bedrooms` coefficient for the new model is insignificant, so we don’t need to be too concerned.

Other estimated coefficients such as `bathrooms`, `sqft_living`, and `sqft_lot` also change by large percentages.

We now need to assess whether the goals of our analysis are practically affected by our influential observations.

-   If our goal is a general understanding of the relationship between the response and regressors, then even though the estimated coefficients substantially changed in some cases, the overall associations (positive or negative) have been preserved (taking into account that some coefficients are insignificant).
-   If our goal is to assess the quantitative association between a regressor and the response, then our influential observations dramatically affect our analysis goal because our estimates are substantially different. However, this may not be a sensible goal since there can always be lurking variables that aren’t accounted for in an observational study that could impact the association between the response and regressors.
-   If our goal is to predict new observations, then we will need to determine whether the influential observations substantially change our predicted response.

We will predict the sales `price` of a “typical” home below to see the impact of the influential observations.

We starting be creating a data frame of predictor values for a typical home.

In [ ]:
newdf <- data.frame(bedrooms = 3, bathrooms = 2, sqft_living = 2000,
                    sqft_lot = 6000, floors = 1, waterfront = "no",
                    condition = "average")

We use the `predict` function to compute the predict `sales` price of the typical home for our original fitted model and the model that removes the influential observations.

In [ ]:
predict(lmod, newdata = newdf)

In [ ]:
predict(lmod2, newdata = newdf)

In this case, the predicted value of `price` increased by 0.2% when we removed the influential observations from our analysis.

Since `price` is in log base 10 units, this amounts to a 2.8% increase on the original scale.

On either scale, the influential observations may not impact the predicted response enough for us to be concerned.

**What if our influential observations are problematic?**

What if certain influential observations can’t be removed from analysis and negatively impact the goals of our analysis?

-   Influential observations are model specific. We may need to address structural issues with our model to reduce the impact of our influential observations.
-   We may need to use an alternative to ordinary least squares regression such as robust regression (see the `rlm` function in the **MASS** package via `?MASS::rlm`) to mitigate the influence of the influential observations. Robust regression is designed to reduce the impact of extreme observations on the fitted model.

# Summary of methods

**Outlier detection**

-   Index plot of studentized residuals.
    -   Compare residuals to the thresholds $\pm t^{\alpha/2n}_{p, n-p}$.
    -   Use the `outlier_plot` function in the **api2lm** package.
-   Outlier test based on the $t$ distribution.
    -   Use the `outlier_test` function in the **api2lm** package.

**Leverage point detection**

-   Index plot of leverage values.
    -   Compare leverage values to the threshold 0.5 or $2p/n$.
    -   Use the `leverage_plot` function in the **api2lm** package.

**Influential observation detection**

-   Index plots of DFBETAS statistics.
    -   Compare the DFBETAS statistics to the thresholds $\pm 1$.
    -   Use the `dfbetas_plot` function in the **api2lm** package.
-   Index plot of DFFITS statistics.
    -   Compare the DFFITS statistics to the thresholds $\pm 2\sqrt{p/n}$.
    -   Use the `dffits_plot` function in the **api2lm** package.
-   Index plot of Cook’s distances.
    -   Compare the Cook’s distances to the threshold $F^{0.5}_{p, n-p}$.
    -   Use the `cooks_plot` function in the **api2lm** package.
-   Influence plot.
    -   Look for observations with large residuals, leverage values, or Cook’s distance/DFFITS statistics.
    -   Use the `influence_plot` function in the **api2lm** package.

**Additional influence-related R functions**

-   `rstudent(lmod)` extracts the studentized residuals from a fitted model.
-   `hatvalues(lmod)` extracts the leverage values from a fitted model.
-   `dfbetas` extracts the DFBETAS matrix from a fitted model.
-   `dffits` extracts the DFFITS statistics from a fitted model.
-   `cooks.distance` extracts the Cook’s distances from a fitted model.
-   `influence(lmod)` computes several leave-one-out-related measures of observational influence.

# References

Belsley, David A, Edwin Kuh, and Roy E Welsch. 2005. *Regression Diagnostics: Identifying Influential Data and Sources of Collinearity*. John Wiley & Sons.

Cook, R. Dennis. 1977. “Detection of Influential Observation in Linear Regression.” *Technometrics* 19 (1): 15–18.

Kutner, Michael H, Christopher J Nachtsheim, John Neter, and William Li. 2005. *Applied Linear Statistical Models, 5th Edition*. McGraw-Hill/Irwin, New York.

Welsch, Roy E, and Edwin Kuh. 1977. “Linear Regression Diagnostics.” National Bureau of Economic Research.